In [1]:
import os
from typing import Union
import PyPDF2

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain_core.exceptions import OutputParserException

from neo4j import GraphDatabase
from enum import Enum

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
class DocClass(Enum):
    RESUME = "resume"
    SCIENCE_ARTICLE = "science_article"

In [3]:
class Neo4jConnection:
    def __init__(self, uri, username, password, database="neo4j"):
        self.uri = uri
        self.username = username
        self.password = password
        self.database = database
        self.driver = None
        
        try:
            self.driver = GraphDatabase.driver(self.uri, auth=(self.username, self.password))
            print("Connection to database successful")
        
        except Exception as e:
            print(e)
            
    def close(self):
        if self.driver is not None:
            self.driver.close()
            print("Neo4j connection closed")
            
    def query(self, query, parameters=None):
        
        assert self.driver is not None, "driver is not initialized!"
        session = None
        response = None
        
        try:
            session = self.driver.session(database=self.database)
            response = list(session.run(query, parameters or {}))
            
        except Exception as e:
            print(e)
            
        return response
    
    def write_transaction(self, query, parameters=None):
        assert self.driver is not None, "driver is not initialized!"
        session = None
        
        try: 
            session = self.driver.session(database=self.database)
            session.execute_write(lambda tx: tx.run(query, parameters or {}))
            
        except Exception as e:
            print(e)

In [4]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [5]:
print(NEO4J_URI)
print(NEO_USERNAME)
print(NEO4J_PASSWORD)

bolt://localhost:7687
neo4j
Admin@123


In [6]:
neo4j_connection = Neo4jConnection(NEO4J_URI, NEO_USERNAME,NEO4J_PASSWORD)

Connection to database successful


In [15]:
doc_class = "SCIENCE_ARTICLE"
query = f"MERGE (r:{doc_class}{{name: 'technical_document.pdf'}}) RETURN r"

In [16]:
neo4j_connection.write_transaction(query)

In [18]:
path = "knowledge_graphs/docs/Hasnain Ali Resume.pdf"
full_text = ""
with open(path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    for page in reader.pages:
        full_text += page.extract_text()

In [19]:
full_text

'HASNAIN ALI POONJA  \n    Address:  201, Adam  Palace  Garden  East Karachi.  \n   Contact  No: +92 300 8921854  \n   LinkedIn:  https://www.linkedin.com/in/hasnain -ali-poonja/  \n  Gmail:  hasnainali.ha659@gmail.com  \n  GitHub:  https://github.com/hasnainali659  \n  Medium:  https://medium.com/@hasnainali.ha659  \n \n \n   Objective  \nDynamic and results -driven Machine Learning Engineer with a proven track record of success in \nleading AI product and projects . Seeking a challenging position at a cutting -edge tech company to \nleverage my skills in developing and deploying advanced machine learning models that contribute \nsignificantly to business growth.  \n \nEducation  \n \nMS Robotics and Artificial Intelligence , NUST SMME, Islamabad.  \nSpecialization (Deep Learning , Computer Vision  and Natural Language Processing ) \nCGPA 3.81  \n \nBE Mechanical Engineer , NED University , Karachi .  \nSpecialization (Robotics ) \n CGPA 3. 63 \n \n  Experience  \n \n• 12/202 3 – Pres

In [29]:
def get_all_nodes_and_relationships():
    
    NEO4J_URI = os.getenv("NEO4J_URI")
    NEO_USERNAME = os.getenv("NEO4J_USERNAME")
    NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
    
    query = """MATCH (n) OPTIONAL MATCH (n)-[r]-() RETURN DISTINCT n, r"""
    
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO_USERNAME, NEO4J_PASSWORD))
    
    nodes = []
    relationships = []
    
    with driver.session() as session:
        results = session.run(query)
        
        for rec in results:
            node = rec["n"]
            relatationship = rec["r"]
            
            if node and hasattr(node, '_properties'):
                node_name = node._properties.get('name')
                
                if node_name:
                    nodes.append(node_name)
            
            if relatationship is not None and hasattr(relatationship, 'type'):
                relationships.append(relatationship)
    
    return nodes, relationships

all_nodes, all_relationships = get_all_nodes_and_relationships()
                

In [30]:
all_relationships

[<Relationship element_id='5:8b745461-b27e-4372-a3af-32f1c679e0c0:1152928101676613633' nodes=(<Node element_id='4:8b745461-b27e-4372-a3af-32f1c679e0c0:1' labels=frozenset({'File'}) properties={'name': 'ayaan.pdf'}>, <Node element_id='4:8b745461-b27e-4372-a3af-32f1c679e0c0:0' labels=frozenset({'resume'}) properties={'name': 'resume'}>) type='BELONGS_TO' properties={}>,
 <Relationship element_id='5:8b745461-b27e-4372-a3af-32f1c679e0c0:1152928101676613639' nodes=(<Node element_id='4:8b745461-b27e-4372-a3af-32f1c679e0c0:7' labels=frozenset({'File'}) properties={'name': 'Khawer-CV.pdf'}>, <Node element_id='4:8b745461-b27e-4372-a3af-32f1c679e0c0:0' labels=frozenset({'resume'}) properties={'name': 'resume'}>) type='BELONGS_TO' properties={}>,
 <Relationship element_id='5:8b745461-b27e-4372-a3af-32f1c679e0c0:1152928101676613633' nodes=(<Node element_id='4:8b745461-b27e-4372-a3af-32f1c679e0c0:1' labels=frozenset({'File'}) properties={'name': 'ayaan.pdf'}>, <Node element_id='4:8b745461-b27e-4372

In [31]:
template = """
IMPORTANT: Ensure your output is valid JSON with all keys and string values enclosed in double quotes.
You are a document parser. Your primary objective is to analyze the provided document and extract key components to construct
a knowledge graph. You will be given the following inputs:

- A document text (which could be a resume, science article, or technical document).
- The document class (which indicates the type or category of the document).
- A list of all previously extracted relationships (all_relationships). For first document, all_relationships will be empty.

Your task is to extract:
1. **Entities:** Key names, institutions, concepts, topics, skills, locations, etc.
2. **Relationships:** The connections between these entities. When determining a relationship, check the provided all_relationships list. 
If a similar relationship already exists (even if represented with different synonyms), use the existing relationship name.  
- *For example:*  
    - **HAS_AUTHORED** and **AUTHORED_BY** are considered the same.  
    - **HAS_CONTRIBUTED** and **AUTHORED_BY** are considered the same.  
    - So, if **HAS_AUTHORED** is already in the all_relationships list, do not create **AUTHORED_BY**; instead, use **HAS_AUTHORED**.  
    - Similarly, if **HAS_CONTRIBUTED** exists, do not generate **AUTHORED_BY**; use **HAS_CONTRIBUTED**.
3. **Cypher Queries:** Generate Cypher queries that logically connect the extracted entities. Each query should:
- Create or merge nodes for each entity.
- Create relationships connecting the nodes, ensuring every entity is linked to the designated root entity.
- Be written in a clear, easy-to-understand manner.
4. **Root Entity Name:** Identify and assign the main or most representative entity of the document as the root node. Every other entity should be connected directly or indirectly to this root.

**Important Guidelines:**
- **Comprehensiveness:** Extract as many entities and relationships as possible. Ensure no relevant piece of information is omitted.
- **Context Sensitivity:** 
- For resumes, focus on aspects like Person Name, Education (degrees or institutions), Work Experience (companies and job titles), Skills, Location, Certifications, Awards, etc.
- For science articles, look for Topic, Authors, Affiliations, Contributions, Novelty, Research methods, Methodology, Results, and Conclusion.
- For technical documents, capture the Topic, Technical Details, How it Works, Applications, and Impact.
- **Connection Logic:**  

- Every node must be connected to the root entity node via an appropriate relationship.
- For example, if "NED University" is mentioned, it should be connected with a relationship related to education or institution, not one meant for contributions.  
- Similarly, "Karachi" should be connected with a relationship related to location or residence rather than an institution-related relationship.

**Output Format:**
Use the following JSON-like structure as a guide. Do not alter the double curly braces {{ }} as they are required by the Langchain format.
{{
    'entities': [...],
    'relationships': [...],
    'cypher_queries': [...],
    'root_entity_name': '...'
}}

**Detailed Examples:**

----------------------------------
Example 1: Resume Document
----------------------------------
Document:
"My name is Alice Johnson. I have a Bachelor of Science in Computer Science from the University of Texas. I worked at IBM as a Data Scientist. My skill set includes Python, Machine Learning, and Data Analysis."

Expected Output:
{{
    'entities': ['Alice Johnson', 'University of Texas', 'IBM', 'Data Scientist', 'Python', 'Machine Learning', 'Data Analysis'],
    'relationships': ['HAS_EDUCATION', 'HAS_EXPERIENCE', 'HAS_SKILLS'],
    'cypher_queries': [
        "MERGE (person:Entity {{name: 'Alice Johnson'}}) RETURN person",
        "MERGE (school:Entity {{name: 'University of Texas'}}) RETURN school",
        "MERGE (company:Entity {{name: 'IBM'}}) RETURN company",
        "MERGE (role:Entity {{name: 'Data Scientist'}}) RETURN role",
        "MERGE (skill1:Entity {{name: 'Python'}}) RETURN skill1",
        "MERGE (skill2:Entity {{name: 'Machine Learning'}}) RETURN skill2",
        "MERGE (skill3:Entity {{name: 'Data Analysis'}}) RETURN skill3",
        "MATCH (person:Entity {{name: 'Alice Johnson'}}), (school:Entity {{name: 'University of Texas'}}) MERGE (person)-[:HAS_EDUCATION]->(school)",
        "MATCH (person:Entity {{name: 'Alice Johnson'}}), (company:Entity {{name: 'IBM'}}) MERGE (person)-[:HAS_EXPERIENCE]->(company)",
        "MATCH (person:Entity {{name: 'Alice Johnson'}}), (role:Entity {{name: 'Data Scientist'}}) MERGE (person)-[:HAS_EXPERIENCE]->(role)",
        "MATCH (person:Entity {{name: 'Alice Johnson'}}), (skill1:Entity {{name: 'Python'}}) MERGE (person)-[:HAS_SKILLS]->(skill1)",
        "MATCH (person:Entity {{name: 'Alice Johnson'}}), (skill2:Entity {{name: 'Machine Learning'}}) MERGE (person)-[:HAS_SKILLS]->(skill2)",
        "MATCH (person:Entity {{name: 'Alice Johnson'}}), (skill3:Entity {{name: 'Data Analysis'}}) MERGE (person)-[:HAS_SKILLS]->(skill3)"
    ],
    'root_entity_name': 'Alice Johnson'
}}

----------------------------------
Example 2: Resume with Additional Details
----------------------------------
Resume:
"I am Bob Smith, living in San Francisco. I graduated with a Master's in Data Science from Stanford University, and I have experience at Google as a Machine Learning Engineer. I am proficient in Python, C++, and SQL. I also received the 'Innovator Award' for my contributions to AI research."

Expected Output:
{{
    'entities': ['Bob Smith', 'San Francisco', 'Stanford University', 'Google', 'Machine Learning Engineer', 'Python', 'C++', 'SQL', 'Innovator Award', 'AI Research'],
    'relationships': ['LIVES_IN', 'HAS_EDUCATION', 'HAS_EXPERIENCE', 'HAS_SKILLS', 'HAS_AWARDS', 'HAS_RESEARCH'],
    'cypher_queries': [
        "MERGE (person:Entity {{name: 'Bob Smith'}}) RETURN person",
        "MERGE (city:Entity {{name: 'San Francisco'}}) RETURN city",
        "MERGE (university:Entity {{name: 'Stanford University'}}) RETURN university",
        "MERGE (company:Entity {{name: 'Google'}}) RETURN company",
        "MERGE (role:Entity {{name: 'Machine Learning Engineer'}}) RETURN role",
        "MERGE (skill1:Entity {{name: 'Python'}}) RETURN skill1",
        "MERGE (skill2:Entity {{name: 'C++'}}) RETURN skill2",
        "MERGE (skill3:Entity {{name: 'SQL'}}) RETURN skill3",
        "MERGE (award:Entity {{name: 'Innovator Award'}}) RETURN award",
        "MERGE (research:Entity {{name: 'AI Research'}}) RETURN research",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (city:Entity {{name: 'San Francisco'}}) MERGE (person)-[:LIVES_IN]->(city)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (university:Entity {{name: 'Stanford University'}}) MERGE (person)-[:HAS_EDUCATION]->(university)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (company:Entity {{name: 'Google'}}) MERGE (person)-[:HAS_EXPERIENCE]->(company)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (role:Entity {{name: 'Machine Learning Engineer'}}) MERGE (person)-[:HAS_EXPERIENCE]->(role)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (skill1:Entity {{name: 'Python'}}) MERGE (person)-[:HAS_SKILLS]->(skill1)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (skill2:Entity {{name: 'C++'}}) MERGE (person)-[:HAS_SKILLS]->(skill2)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (skill3:Entity {{name: 'SQL'}}) MERGE (person)-[:HAS_SKILLS]->(skill3)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (award:Entity {{name: 'Innovator Award'}}) MERGE (person)-[:HAS_AWARDS]->(award)",
        "MATCH (person:Entity {{name: 'Bob Smith'}}), (research:Entity {{name: 'AI Research'}}) MERGE (person)-[:HAS_RESEARCH]->(research)"
    ],
    'root_entity_name': 'Bob Smith'
}}

----------------------------------
Example 3: Science Article
----------------------------------
Document:
"The article discusses the latest advancements in robotics, including the development of a new robotic arm that can perform complex tasks. The research highlights experimental methodologies and significant improvements in precision."

Expected Output:
{{
    'entities': ['Robotics', 'Robotic Arm', 'Complex Tasks', 'Experimental Methodologies', 'Precision Improvements'],
    'relationships': ['HAS_ADVANCES', 'HAS_DEVELOPMENT', 'HAS_METHODOLOGY', 'HAS_RESULTS'],
    'cypher_queries': [
        "MERGE (article:Entity {{name: 'Robotics Article'}}) RETURN article",
        "MERGE (advancement:Entity {{name: 'Robotics'}}) RETURN advancement",
        "MERGE (arm:Entity {{name: 'Robotic Arm'}}) RETURN arm",
        "MERGE (task:Entity {{name: 'Complex Tasks'}}) RETURN task",
        "MERGE (method:Entity {{name: 'Experimental Methodologies'}}) RETURN method",
        "MERGE (result:Entity {{name: 'Precision Improvements'}}) RETURN result",
        "MATCH (article:Entity {{name: 'Robotics Article'}}), (advancement:Entity {{name: 'Robotics'}}) MERGE (article)-[:HAS_ADVANCES]->(advancement)",
        "MATCH (article:Entity {{name: 'Robotics Article'}}), (arm:Entity {{name: 'Robotic Arm'}}) MERGE (article)-[:HAS_DEVELOPMENT]->(arm)",
        "MATCH (article:Entity {{name: 'Robotics Article'}}), (method:Entity {{name: 'Experimental Methodologies'}}) MERGE (article)-[:HAS_METHODOLOGY]->(method)",
        "MATCH (article:Entity {{name: 'Robotics Article'}}), (result:Entity {{name: 'Precision Improvements'}}) MERGE (article)-[:HAS_RESULTS]->(result)"
    ],
    'root_entity_name': 'Robotics'
}}

----------------------------------
Example 4: Technical Document
----------------------------------
Document:
"This Handbook is a comprehensive guide to the latest advancements in AI, including the development of a new AI model that can perform complex tasks. AI is the future of the world, influencing industries and transforming societies."

Expected Output:
{{
    'entities': ['AI', 'AI Model', 'Complex Tasks', 'Future of the World', 'Industries', 'Societies'],
    'relationships': ['HAS_ADVANCES', 'HAS_DEVELOPMENT', 'HAS_IMPACT', 'HAS_APPLICATION'],
    'cypher_queries': [
        "MERGE (document:Entity {{name: 'Handbook'}}) RETURN document",
        "MERGE (topic:Entity {{name: 'AI'}}) RETURN topic",
        "MERGE (model:Entity {{name: 'AI Model'}}) RETURN model",
        "MERGE (task:Entity {{name: 'Complex Tasks'}}) RETURN task",
        "MERGE (impact:Entity {{name: 'Future of the World'}}) RETURN impact",
        "MERGE (industry:Entity {{name: 'Industries'}}) RETURN industry",
        "MERGE (society:Entity {{name: 'Societies'}}) RETURN society",
        "MATCH (document:Entity {{name: 'Handbook'}}), (topic:Entity {{name: 'AI'}}) MERGE (document)-[:HAS_ADVANCES]->(topic)",
        "MATCH (document:Entity {{name: 'Handbook'}}), (model:Entity {{name: 'AI Model'}}) MERGE (document)-[:HAS_DEVELOPMENT]->(model)",
        "MATCH (document:Entity {{name: 'Handbook'}}), (task:Entity {{name: 'Complex Tasks'}}) MERGE (document)-[:HAS_DEVELOPMENT]->(task)",
        "MATCH (document:Entity {{name: 'Handbook'}}), (impact:Entity {{name: 'Future of the World'}}) MERGE (document)-[:HAS_IMPACT]->(impact)",
        "MATCH (document:Entity {{name: 'Handbook'}}), (industry:Entity {{name: 'Industries'}}) MERGE (document)-[:HAS_APPLICATION]->(industry)",
        "MATCH (document:Entity {{name: 'Handbook'}}), (society:Entity {{name: 'Societies'}}) MERGE (document)-[:HAS_APPLICATION]->(society)"
    ],
    'root_entity_name': 'AI'
}}

----------------------------------
Example 5: Mixed Technical and Research Document
----------------------------------
Document:
"Dr. Emily Clark presents a whitepaper on quantum computing. The document outlines the principles of quantum mechanics, details the design of a quantum processor, and discusses the potential impact on cybersecurity and data encryption. It also covers experimental results from recent trials."

Expected Output:
{{
    'entities': ['Dr. Emily Clark', 'Quantum Computing', 'Quantum Mechanics', 'Quantum Processor', 'Cybersecurity', 'Data Encryption', 'Experimental Results'],
    'relationships': ['HAS_AUTHORED', 'HAS_TOPICS', 'HAS_DEVELOPMENT', 'HAS_IMPACT', 'HAS_RESULTS'],
    'cypher_queries': [
        "MERGE (document:Entity {{name: 'Whitepaper on Quantum Computing'}}) RETURN document",
        "MERGE (author:Entity {{name: 'Dr. Emily Clark'}}) RETURN author",
        "MERGE (topic:Entity {{name: 'Quantum Computing'}}) RETURN topic",
        "MERGE (mechanics:Entity {{name: 'Quantum Mechanics'}}) RETURN mechanics",
        "MERGE (processor:Entity {{name: 'Quantum Processor'}}) RETURN processor",
        "MERGE (cyber:Entity {{name: 'Cybersecurity'}}) RETURN cyber",
        "MERGE (encryption:Entity {{name: 'Data Encryption'}}) RETURN encryption",
        "MERGE (results:Entity {{name: 'Experimental Results'}}) RETURN results",
        "MATCH (document:Entity {{name: 'Whitepaper on Quantum Computing'}}), (author:Entity {{name: 'Dr. Emily Clark'}}) MERGE (document)-[:HAS_AUTHORED]->(author)",
        "MATCH (document:Entity {{name: 'Whitepaper on Quantum Computing'}}), (topic:Entity {{name: 'Quantum Computing'}}) MERGE (document)-[:HAS_TOPICS]->(topic)",
        "MATCH (document:Entity {{name: 'Whitepaper on Quantum Computing'}}), (mechanics:Entity {{name: 'Quantum Mechanics'}}) MERGE (document)-[:HAS_TOPICS]->(mechanics)",
        "MATCH (document:Entity {{name: 'Whitepaper on Quantum Computing'}}), (processor:Entity {{name: 'Quantum Processor'}}) MERGE (document)-[:HAS_DEVELOPMENT]->(processor)",
        "MATCH (document:Entity {{name: 'Whitepaper on Quantum Computing'}}), (cyber:Entity {{name: 'Cybersecurity'}}) MERGE (document)-[:HAS_IMPACT]->(cyber)",
        "MATCH (document:Entity {{name: 'Whitepaper on Quantum Computing'}}), (encryption:Entity {{name: 'Data Encryption'}}) MERGE (document)-[:HAS_IMPACT]->(encryption)",
        "MATCH (document:Entity {{name: 'Whitepaper on Quantum Computing'}}), (results:Entity {{name: 'Experimental Results'}}) MERGE (document)-[:HAS_RESULTS]->(results)"
    ],
    'root_entity_name': 'Quantum Computing'
}}

----------------------------------
Instructions for Processing the Input:
Now, using the same approach and output format structure shown in the examples above, parse the following inputs:

- **Document:** {text}
- **Document Class:** {doc_class}
- **All Relationships:** {all_relationships}


Your output should comprehensively list all relevant entities, determine the appropriate relationships (reusing existing relationship names when applicable), and generate clear, logically connected cypher queries that integrate every extracted entity with the chosen root entity. Make sure no entity is left unconnected.

Remember:  
- Use clear and descriptive relationship names that reflect the nature of the connection (e.g., HAS_EDUCATION, LIVES_IN, HAS_EXPERIENCE, HAS_ADVANCES, HAS_DEVELOPMENT, HAS_IMPACT, etc.).
"""

In [32]:
llm = ChatOpenAI(model="gpt-4o")

In [33]:
print(llm.invoke("hi"))

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f33640a400', 'id': 'chatcmpl-CAHalLntyhPky0reNqbULE2M8O88N', 'finish_reason': 'stop', 'logprobs': None} id='run--f548db96-6cc1-4c8c-8f12-5ace3e5a348f-0' usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [34]:
from pydantic import BaseModel, Field

class ContentSchema(BaseModel):
    entities: list[str] = Field(default=[], description="The list of entities in resume")
    relationships: list[str] = Field(default=[], description="The list of relationships between entities of resume")
    cypher_queries: list[str] = Field(default=[], description="The list of cypher queries to create a knowlege graph")
    root_entity_name: str = Field(default="", description="The name of root entity e.g Hasnain, Bob Smith")


In [35]:
parser = PydanticOutputParser(pydantic_object=ContentSchema)

In [36]:
prompt = PromptTemplate.from_template(template)
chain = prompt | llm

In [37]:
doc_class = "resume"

In [38]:
response = chain.invoke({"text": full_text, "doc_class": doc_class, "all_relationships": all_relationships})

In [40]:
parsed_response = parser.parse(response.content)

In [45]:
entities = parsed_response.entities
relationships = parsed_response.relationships
cypher_queries = parsed_response.cypher_queries
root_entity_node = parsed_response.root_entity_name

In [46]:
file_name = "Hasnain Ali Resume"
file_query = """MERGE (f: File {name: $file_name}) RETURN f"""
file_query_params = {'file_name': file_name}

neo4j_connection.write_transaction(file_query, file_query_params)

In [51]:
doc_class

'resume'

In [52]:
belongs_query = f"""
    MATCH (r:{doc_class} {{ name: '{doc_class.lower()}' }}), (f:File {{ name: $file_name }})
    MERGE (f)-[:BELONGS_TO]->(r) 
    RETURN r, f
"""
neo4j_connection.write_transaction(belongs_query, file_query_params)

In [53]:
for cypher_query in cypher_queries:
    neo4j_connection.write_transaction(cypher_query)

In [54]:
root_entity_query = """
    MATCH (b { name: $root_entity_name }), (f { name: $file_name })
    MERGE (b)-[:HAS_FILE]->(f)
    RETURN b, f
"""

In [55]:
root_entity_query_params = {
    'root_entity_name': root_entity_node,
    'file_name': file_name
}

In [56]:
neo4j_connection.write_transaction(root_entity_query, root_entity_query_params)